# 7 - Manejo de Datos: Union, Combinación y Remodelación
======================================================

#### IPN

#### 20/nov/2020

-   [Indexación jerárquica](#indexación-jerárquica)
    -   [Reordenamiento y clasificación de
        niveles](#reordenamiento-y-clasificación-de-niveles)
    -   [Resumen de estadísticas por
        nivel](#resumen-de-estadísticas-por-nivel)
    -   [Indexación con columnas de un
        DataFrame](#indexación-con-columnas-de-un-dataframe)
-   [Combinación y unión de conjuntos de
    datos](#combinación-y-unión-de-conjuntos-de-datos)
    -   [Unión de DataFrames
        Tipo-Database](#unión-de-dataframes-tipo-database)
    -   [Unión con Índice](#unión-con-índice)
    -   [Concatenación a lo Largo de un
        Eje](#concatenación-a-lo-largo-de-un-eje)
    -   [Combinación de Datos con
        Superposición](#combinación-de-datos-con-superposición)
-   [Reformateo](#reformateo)
    -   [Remodelación con Indexación
        Jerárquica](#remodelación-con-indexación-jerárquica)
    -   [Pivotar Formato “Largo” a
        “Ancho”](#pivotar-formato-largo-a-ancho)
    -   [Pivotar Formato “Ancho” a
        “Largo”](#pivotar-formato-ancho-a-largo)
-   [Bibliografía](#bibliografía)

 

 

 

NOTA: *Los archivos examples y datasets se pueden obtener del sitio
<a href="https://github.com/wesm/pydata-book" class="uri">https://github.com/wesm/pydata-book</a>*

 

En muchas aplicaciones, los datos pueden extenderse a través de varios
archivos o bases de datos o pueden organizarse de una forma que no sea
fácil de analizar. Este capítulo se centra en las herramientas para
ayudar a combinar, unir y reorganizar datos. Iniciaremos con el concepto
de indexación jerárquica en pandas, que se usa ampliamente en algunas de
estas operaciones. Luego profundizaremos en las manipulaciones de datos
particulares.

 

 

 

Indexación jerárquica
---------------------

 

La *indexación jerárquica* es una característica importante de pandas
que permite tener múltiples (dos o más) niveles de índice en un eje. De
manera algo abstracta, proporciona una forma de trabajar con datos de
dimensiones superiores en una forma de dimensiones inferiores.

Comencemos con un ejemplo simple; creamos una Serie con una lista de
listas (o arreglos) como índice:

 

  
    

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.Series(np.random.randn(9),
                     index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                            [1, 2, 3, 1, 3, 1, 2, 2, 3]])

In [ ]:
 data

Con los datos indexados jerárquicamente, las cosas son más complicadas,
ya que unir con índice es implícitamente una unión de claves-múltiples:


In [ ]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
    ...:                       'key2': [2000, 2001, 2002, 2001, 2002],
    ...:                       'data': np.arange(5.)})

Estamos viendo una vista de una Serie con un `MultiIndex` como índice.
Los “espacios” en la visualización del índice significan “usar la
etiqueta que está directamente arriba”:
 

In [ ]:
data.index

Con un objeto indexado jerárquicamente, es posible la denominada
indexación parcial, que permite seleccionar de manera concisa
subconjuntos de datos:
  

In [ ]:
data['b']

In [ ]:
data['b':'c']

In [ ]:
data.loc[['b', 'd']]

La selección es incluso posible desde un nivel “interno”:

In [ ]:
data.loc[:, 2]

La indexación jerárquica juega un papel importante en la remodelación de
datos y operaciones basadas en grupos, como la formación de una tabla
dinámica. Por ejemplo, podría reorganizar los datos en un DataFrame
utilizando su método `unstack`:

In [ ]:
data.unstack()

La operación inversa de `unstack` es `stack`:

In [ ]:
data.unstack().stack()

Con un DataFrame, cualquier eje puede tener un índice jerárquico:

In [ ]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                           index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                           columns=[['Ohio', 'Ohio', 'Colorado'],
                                    ['Green', 'Red', 'Green']])


In [ ]:
frame

Los niveles jerárquicos pueden tener nombres (como cadenas o cualquier
objeto de Python). Si es así, estos aparecerán en la salida de la
consola:

In [ ]:
frame.index.names = ['key1', 'key2']

In [ ]:
frame.columns.names = ['state', 'color']

In [ ]:
frame

De manera similar, con la indexación parcial de columnas podemos
seleccionar grupos de columnas:

In [ ]:
frame['Ohio']

Se puede crear un `MultiIndex` por sí mismo y luego reutilizarlo; las
columnas en el DataFrame anterior con nombres de nivel podrían crearse
así:

In [ ]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
    ...:                           names=['state', 'color'])

### Reordenamiento y clasificación de niveles

 

En ocasiones, deberá reorganizar el orden de los niveles en un eje u
ordenar los datos por los valores en un nivel específico. `swaplevel`
toma dos nombres o números de nivel y retorna un nuevo objeto con los
niveles intercambiados  —–pero los datos no son alterados:

 

In [ ]:
frame.swaplevel('key1', 'key2')

Por otro lado, `sort_index` ordena los datos utilizando solo los valores
en un solo nivel. Al intercambiar niveles, no es raro usar también
`sort_index` para que el resultado se clasifique lexicográficamente por
el nivel indicado:

In [ ]:
frame.sort_index(level=1)

In [ ]:
frame.swaplevel(0, 1).sort_index(level=0)

### Resumen de estadísticas por nivel

 

Muchas estadísticas descriptivas y de resumen en DataFrame y Series
tienen una opción `level` en el que puede especificar el nivel que desea
agregar en un eje en particular. Consideremos el DataFrame anterior;
podemos agregar por nivel en las filas o columnas de la siguiente
manera:

In [ ]:
frame.sum(level='key2')

In [ ]:
frame.sum(level='color', axis=1)

### Indexación con columnas de un DataFrame

No es inusual querer usar una o más columnas de un DataFrame como índice
de fila; alternativamente, es posible que desee mover el índice de la
fila a las columnas del Marco de datos. Aquí hay un ejemplo de
DataFrame:


In [ ]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
    ...:                       'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
    ...:                       'd': [0, 1, 2, 0, 1, 2, 3]})

In [ ]:
frame

La función set\_index\` de DataFrame creará un nuevo DataFrame
utilizando una o más de sus columnas como índice:
 

In [ ]:
frame2 = frame.set_index(['c', 'd'])

In [ ]:
frame2

Por “default”, las columnas se eliminan del DataFrame, aunque podemos
dejarlas:

In [ ]:
frame.set_index(['c', 'd'], drop=False)

por otro lado, `reset_index`, hace lo contrario de `set_index`; los
niveles de índice jerárquico se mueven a las columnas:
   

In [ ]:
frame2.reset_index()

Combinación y unión de conjuntos de datos
-----------------------------------------

 

Los datos contenidos en los objetos pandas se pueden combinar de varias
maneras:

-   `pandas.merge` conecta filas en DataFrames en función de una o más
    claves. Esto es familiar para los usuarios de SQL u otras bases de
    datos relacionales, ya que implementa operaciones de *unión* en
    bases de datos.

-   `pandas.concat` concatena o “apila” objetos juntos a lo largo de un
    eje.

-   El método de `instancia combine_first` permite unir datos
    superpuestos para completar los valores faltantes en un objeto con
    valores de otro.

 

 

### Unión de DataFrames Tipo-Database

 

Las operaciones *merge* o *unión* combinan conjuntos de datos al
vincular filas con una o más *claves*. Estas operaciones son centrales
para las bases de datos relacionales (por ejemplo, basadas en SQL). La
función `merge` en pandas es el principal punto de entrada para usar
estos algoritmos en sus datos.

Comencemos con un ejemplo simple:
 

In [ ]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                          'data1': range(7)})

In [ ]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                          'data2': range(3)})

In [ ]:
df1

In [ ]:
df2

Este es un ejemplo de una unión de *muchos a uno*; los datos en `df1`
tienen varias filas etiquetadas con `a` y `b`, mientras que `df2` tiene
solo una fila para cada valor en la columna clave. Llamando a `merge`
con estos objetos obtenemos:
 

In [ ]:
pd.merge(df1, df2)

En el ejemplo anterior no especifiamos en qué columna unir. Si no se
especifica esa información, `merge` usa como claves los nombres de
columna superpuestos . Sin embargo, es una buena práctica especificar
explícitamente:
 

In [ ]:
pd.merge(df1, df2, on='key')

Si los nombres de columna son diferentes en cada objeto, podemos
especificarlos por separado:
 

In [ ]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
    ...:                     'data1': range(7)})

In [ ]:
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
    ...:                     'data2': range(3)})

In [ ]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

Notamos que en el resultado faltan los valores `'c'` y `'d'` y los datos
asociados. Por defecto `merge` hace una unión `'inner'`; Las claves en
el resultado son la intersección, o el conjunto común que se encuentra
en ambas tablas. Otras opciones posibles son `'left'`, `'right'` y
`'outer'`. La unión outer toma la unión de las teclas, combinando el
efecto de aplicar las uniones izquierda y derecha:
  

In [ ]:
pd.merge(df1, df2, how='outer')

#### Opciones de how

| *Opción* | *Descripción*                                                                                                                  |
|:---------|:-------------------------------------------------------------------------------------------------------------------------------|
| `inner`  | Usa la intersección de claves de ambos frames, similar a una unión interna de SQL; preserva el orden de las claves izquierdas. |
| `left`   | Usa solo las claves del frame izquierdo, similar a una unión externa izquierda de SQL; preserva el orden de las claves.        |
| `outer`  | Usa la unión de claves de ambos frames, similar a una unión externa completa de SQL; ordena las claves lexicográficamente.     |
| `right`  | Usa solo las claves del frame derecho, similar a una unión externa derecha de SQL; preservar el orden de las claves.           |

 

Las uniones de *muchos-a-muchos* tienen un comportamiento bien definido,
aunque no necesariamente intuitivo. Ejemplo:

  

In [ ]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 'data1': range(6)})

In [ ]:
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],  'data2': range(5)})

In [ ]:
df1

In [ ]:
df2

In [ ]:
pd.merge(df1, df2, on='key', how='left')

Las uniones de *muchos-a-muchos* forman el producto Cartesiano de las
filas. Como había tres filas `'b'` en el DataFrame izquierdo y dos en el
derecho, hay seis filas `'b'` en el resultado. El método de combinación
solo afecta a los distintos valores clave que aparecen en el resultado:
 

In [ ]:
pd.merge(df1, df2, how='inner')

Para unir con varias claves, pasamos una lista de nombres de columnas:
 

In [ ]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                           'key2': ['one', 'two', 'one'],
                           'lval': [1, 2, 3]})

In [ ]:
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                           'key2': ['one', 'one', 'one', 'two'],
                             'rval': [4, 5, 6, 7]})

In [ ]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

Para determinar qué combinaciones de claves aparecerán en el resultado
dependiendo de la elección del método de unión (“merge”), piense en las
múltiples claves como formando un arreglo de tuplas que se usan como una
sola clave de unión (aunque en realidad no se implementa de esa manera).

Un último tema a considerar en las operaciones de unión es el
tratamiento de los nombres de columnas superpuestos. Si bien podemos
abordar la superposición manualmente, `merge` tiene una opción
`suffixes` para especificar cadenas que se anexarán a nombres
superpuestos en los objetos DataFrame izquierdo y derecho:

 
 

In [ ]:
pd.merge(left, right, on='key1')

In [ ]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

#### Parámetros de la función merge

| *Argumento*   | *Descripción*                                                                                                                                                                                                                                                                                                                                                                                                        |
|:--------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `copy`        | Si es False, evita copiar si es posible. *“Default”, True*.                                                                                                                                                                                                                                                                                                                                                          |
| `how`         | Tipo de unión a realizar. Ver la tabla anterior de “Opciones de how”. *“Default”, ‘inner’*                                                                                                                                                                                                                                                                                                                           |
| `indicator`   | Si es True, agrega una columna al DataFrame de salida llamado "\_merge" con información sobre el origen de cada fila. Le podemos dar un nombre diferente a la columna proporcionando un argumento de cadena. *“Default”, False*.                                                                                                                                                                                     |
| `left_index`  | Usa el índice del DataFrame izquierdo como la(s) clave(s) de unión. Si es un MultiIndex, la cantidad de claves en el otro DataFrame (ya sea el índice o una cantidad de columnas) debe coincidir con la cantidad de niveles. *“Default”, False*.                                                                                                                                                                     |
| `left_on`     | Nombres de columna o nivel de índice para unirse en el DataFrame izquierdo. También puede ser un arreglo o una lista de arreglos de la longitud del DataFrame izquierdo. Estos arreglos se tratan como si fueran columnas.                                                                                                                                                                                           |
| `on`          | Nombres de columna o nivel de índice para unirse. Estos deben encontrarse en ambos DataFrames. Si *on* es None y no se está fusionanso en los índices, entonces por default se hace la intersección de las columnas en ambos DataFrames.                                                                                                                                                                             |
| `right`       | Objeto con el cual fusionarse.                                                                                                                                                                                                                                                                                                                                                                                       |
| `right_index` | Usa el índice del DataFrame derecho como clave de unión. Las mismas advertencias que left\_index. *“Default”, False*.                                                                                                                                                                                                                                                                                                |
| `right_on`    | Nombres de columna o nivel de índice para unirse en el DataFrame derecho. También puede ser un arreglo o una lista de arreglos de la longitud del DataFrame derecho. Estos arreglos se tratan como si fueran columnas.                                                                                                                                                                                               |
| `sort`        | Ordena las claves de unión lexicográficamente en el DataFrame resultante. Si es False, el orden de las claves de unión depende del tipo de combinación (cer argumento `how`). *“Default”, False*.                                                                                                                                                                                                                    |
| `suffixes`    | Tupla en la que cada elemento es una cadena —opcional— que indica el sufijo que se agregará a los nombres de las columnas superpuestas a la *izquierda* y a la *derecha*, respectivamente. Pase un valor de None en lugar de una cadena para indicar que el nombre de la columna de *izquierda* o *derecha* debe dejarse como está, sin sufijo. Al menos uno de los valores no debe ser None. *“Default” (“x”, “y”*. |

 

 

 

### Unión con Índice

 

En algunos casos, las claves de unión en un DataFrame se encontrarán en
su índice. En este caso, puede pasar `left_index = True` o
`right_index = True` —o ambos— para indicar que el índice debe usarse
como la clave de unión:
 
    

In [ ]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
    ...:                        'value': range(6)})

In [ ]:
   right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [ ]:
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

    

In [ ]:
left1

In [ ]:
right1

In [ ]:
pd.merge(left1, right1, left_on='key', right_index=True)

Dado que por “default” el método de unión es intersecar las claves de
unión, en lugar de eso podemos formar la unión de ellas con una unión
outer:

In [ ]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

Con los datos indexados jerárquicamente, las cosas son más complicadas,
ya que unir con índice es implícitamente una unión de claves-múltiples:

In [ ]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
    ...:                       'key2': [2000, 2001, 2002, 2001, 2002],
    ...:                       'data': np.arange(5.)})

In [ ]:
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
    ...:                       index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'], 
    ...:                              [2001, 2000, 2000, 2000, 2001, 2002]],
    ...:                       columns=['event1', 'event2'])

In [ ]:
lefth

In [ ]:
righth

En este caso, debemos indicar varias columnas para combinar como una
lista (notese el manejo de valores de índice duplicados con
`how = 'outer’`):

In [ ]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

In [ ]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')

También es posible usar los índices de ambos lados de la unión:
 

In [ ]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
    ...:                      index=['a', 'c', 'e'],
    ...:                      columns=['Ohio', 'Nevada'])

In [ ]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
    ...:                       index=['b', 'c', 'd', 'e'],
    ...:                       columns=['Missouri', 'Alabama'])

   

In [ ]:
left2

In [ ]:
right2

In [ ]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

`join` es una instancia muy práctica de DataFrame para fusionar por
índice. También se puede usar para combinar muchos objetos DataFrame que
tienen índices iguales o similares pero columnas no-superpuestas. En el
ejemplo anterior, podríamos haber escrito:

In [ ]:
left2.join(right2, how='outer')

En parte por razones heredadas (es decir, versiones mucho más antiguas
de pandas), el método `join` de DataFrame realiza una unión izquierda en
las claves de unión, preservando exactamente el índice de fila del frame
izquierdo. También admite unirse al índice del DataFrame pasado en una
de las columnas del DataFrame que llama:

In [ ]:
left1.join(right1, on='key')

Por último, para uniones simples de índice sobre índice, podemos pasar
una lista de DataFrames a `join` como alternativa al uso de la función
`concat` más general que se describe en la siguiente sección:
 

In [ ]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                             index=['a', 'c', 'e', 'f'],
                             columns=['New York', 'Oregon'])

   

In [ ]:
another

In [ ]:
left2.join([right2, another])

In [ ]:
left2.join([right2, another], how='outer')

### Concatenación a lo Largo de un Eje

 

Otro tipo de operación de combinación de datos se conoce indistintamente
como concatenación, enlace o apilamiento. La función `concatenate` de
NumPy puede hacer esto con los arreglos de NumPy:

In [ ]:
arr = np.arange(12).reshape((3, 4))

In [ ]:
arr

In [ ]:
np.concatenate([arr, arr], axis=1)

En el contexto de los objetos pandas tales como Series y DataFrame,
tener ejes etiquetados permite generalizar aún más la concatenación de
arreglos. En particular, hay varias posibilidades en las que pensar:

-   Si los objetos se indexan de manera diferente en los otros ejes
    ¿Deberíamos combinar los elementos distintos en estos ejes o usar
    solo los valores compartidos (la intersección)?

-   ¿Los fragmentos de datos concatenados deben ser identificables en el
    objeto resultante?

-   ¿El “eje de concatenación” contiene datos que necesitan ser
    preservados? En muchos casos, las etiquetas de “default” enteras en
    un DataFrame mejor se descartan durante la concatenación.

 

La función `concat` en pandas proporciona una manera consistente de
abordar cada una de estas posibilidades. Veamos una serie de ejemplos
para ilustrar cómo funciona. Supongamos que tenemos tres Series sin
superposición de índice:

In [ ]:
s1 = pd.Series([0, 1], index=['a', 'b'])

In [ ]:
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])

In [ ]:
s3 = pd.Series([5, 6], index=['f', 'g'])

Llamar a `concat` con estos objetos en una lista une los valores y los
índices:

In [ ]:
s1

In [ ]:
pd.concat([s1, s2, s3])

Por “default” `concat` funciona a lo largo de `axis = 0`, produciendo
otra serie. Si pasamos `axis = 1`, el resultado será un DataFrame
(`axis = 1` son las columnas):

In [ ]:
pd.concat([s1, s2, s3], axis=1, sort=False)

En este caso no hay superposición en el otro eje, que como podemos ver
es la unión ordenada (la unión `'outer'`) de los índices. En lugar de
esto, podemos intersecarlos pasando `join = 'inner’`:

In [ ]:
s4 = pd.concat([s1, s3])

In [ ]:
s4

In [ ]:
pd.concat([s1, s4], axis=1, sort=False)

In [ ]:
pd.concat([s1, s4], axis=1, join='inner')

En este último ejemplo, las etiquetas `'f'` y `'g'` desaparecieron
debido a la opción `join = 'inner'`.

Incluso podemos especificar los ejes que se utilizarán en los otros ejes
con el metodo `reidex`:

 

NOTA: El libro usa join\_axes:
`pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])`, página
238 77\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\* QUITAR EN
FINAL\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*

In [ ]:
pd.concat([s1, s4], axis=1, sort=False).reindex(['a', 'c', 'b', 'e'])

Un problema potencial es que las piezas concatenadas no son
identificables en el resultado. Supongamos, en cambio, que desea crear
un índice jerárquico en el eje de concatenación. Para hacer esto, usamos
el argumento `keys`:

In [ ]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])

In [ ]:
result

In [ ]:
result.unstack()

En el caso de combinar Series a lo largo del `axis = 1`, `keys` se
convierte en los encabezados de columna del DataFrame:

In [ ]:
pd.concat([s1, s2, s3], axis=1, sort = True, keys=['one', 'two', 'three'])

La misma lógica se extiende a los objetos DataFrame:

In [ ]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],  columns=['one', 'two'])

In [ ]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],  columns=['three', 'four'])

In [ ]:
df1

In [ ]:
df2

In [ ]:
pd.concat([df1, df2], axis=1, sort = True,  keys=['level1', 'level2'])

Si pasamos un dict de objetos en lugar de una lista, las claves del dict
se usarán para la opción `keys`:

In [ ]:
pd.concat({'level1': df1, 'level2': df2}, axis=1, sort = True)

Existen argumentos adicionales que rigen cómo se crea el índice
jerárquico (siguiente tabla). Por ejemplo, podemos nombrar los niveles
de eje creados con el argumento `names`:
 

In [ ]:
pd.concat([df1, df2], axis=1, sort = True, keys=['level1', 'level2'],
     ...:           names=['upper', 'lower'])

Una última consideración con referencia los DataFrames en los que el
índice de fila no contiene datos relevantes:

In [ ]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])

In [ ]:
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

In [ ]:
df1

In [ ]:
df2

En este caso, podemos pasar `ignore_index = True`:
 

In [ ]:
pd.concat([df1, df2], ignore_index=True, sort = True)

#### Argumentos de la función concat

| *Argumento*        | *Descripción*                                                                                                                                                                                                                                                                                |
|:-------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `axis`             | Eje sobre el cual concatenar.                                                                                                                                                                                                                                                                |
| `copy`             | Si es False no copia datos innecesariamente.                                                                                                                                                                                                                                                 |
| `ignore_index`     | Si es verdadero, no usa los valores de índice a lo largo del eje de concatenación. El eje resultante se etiquetará como 0, …, n - 1.                                                                                                                                                         |
| `join`             | Cómo manejar los índices en otro eje (o ejes).                                                                                                                                                                                                                                               |
| `keys`             | Si pasamos varios niveles, deben contener tuplas. Construye el índice jerárquico usando las claves pasadas como el nivel más externo.                                                                                                                                                        |
| `levels`           | Niveles específicos (valores únicos) a utilizar para construir un MultiIndex. De lo contrario, se deducirán de las claves.                                                                                                                                                                   |
| `objs`             | Si se pasa un dict, las claves ordenadas se utilizarán como el argumento *keys*, a menos que se pase el argumento *keys*, en cuyo caso se seleccionarán los valores. Cualquier objeto None se eliminará silenciosamente a menos que todos sean None, en cuyo caso se generará un ValueError. |
| `names`            | Nombres para los niveles en el índice jerárquico resultante.                                                                                                                                                                                                                                 |
| `sort`             | Ordena el eje de no concatenación si aún no está alineado cuando *join* es “outer”. Esto no tiene efecto cuando `join = 'inner'`, que ya preserva el orden del eje de no concatenación.                                                                                                      |
| `verify_integrity` | Checa si el nuevo eje concatenado contiene duplicados. Esto puede ser muy costoso en relación con la concatenación de datos real.                                                                                                                                                            |

 

 

 

### Combinación de Datos con Superposición

 

Existe otra situación de combinación de datos que no se puede expresar
como una operación de unión o concatenación. Podemos tener dos conjuntos
de datos cuyos índices se superponen en su totalidad o en parte. Como
ejemplo, consideremos la función `where` de NumPy, que realiza el
equivalente orientado a arreglos de una expresión if-else:
 
    

In [ ]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
     ...:                index=['f', 'e', 'd', 'c', 'b', 'a'])

In [ ]:
b = pd.Series(np.arange(len(a), dtype=np.float64),
     ...:               index=['f', 'e', 'd', 'c', 'b', 'a'])

In [ ]:
a

In [ ]:
b

In [ ]:
np.where(pd.isnull(a), b, a)

Las Series tienen un método `combine_first`, que realiza el equivalente
de esta operación junto con la lógica de alineación de datos habitual de
pandas:

In [ ]:
b[:-2].combine_first(a[2:])

Con DataFrames, `combine_first` hace lo mismo columna por columna, por
lo que podemos considerarlo como “parchar” los datos que faltan en el
objeto que llama con los datos del objeto que pasa:

In [ ]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
     ...:                     'b': [np.nan, 2., np.nan, 6.],
     ...:                     'c': range(2, 18, 4)})

In [ ]:
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
     ...:                     'b': [np.nan, 3., 4., 6., 8.]})

In [ ]:
df1

In [ ]:
df2

In [ ]:
df1.combine_first(df2)

Reformateo
----------

 

Existen varias operaciones básicas para reorganizar datos tabulares.
Estos se denominan alternativamente operaciones de *reformato* o
*pivote*.

### Remodelación con Indexación Jerárquica

La indexación jerárquica proporciona una forma consistente de
reorganizar los datos en un DataFrame. Hay dos acciones principales:

-   `stack`     “Rota” o pivota de las columnas en los datos a las
    filas.

-   `unstack`   Pivota de las filas a las columnas.

 

Veamos estas operaciones con una serie de ejemplos. Consideremos un
DataFrame pequeño con arreglos de cadenas como índices de fila y
columna:

In [ ]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                         index=pd.Index(['Ohio', 'Colorado'], name='state'),
                         columns=pd.Index(['one', 'two', 'three'],
                         name='number'))

In [ ]:
data

El método `stack` en estos datos pivota las columnas en las filas,
produciendo una Serie:
 

In [ ]:
result = data.stack()

In [ ]:
result

Desde una serie indexada jerárquicamente, podemos reorganizar los datos
en un DataFrame con `unstack`:

In [ ]:
result.unstack()

Por “default”, el nivel más interno no está apilado (lo mismo con
`stack`). Podemos desapilar un nivel diferente pasando un número o
nombre de nivel:

In [ ]:
result.unstack(0)

In [ ]:
result.unstack('state')

Desapilar podría introducir datos faltantes si no se encuentran todos
los valores en el nivel en cada uno de los subgrupos:

In [ ]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd']) 

In [ ]:
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])

In [ ]:
data2 = pd.concat([s1, s2], keys=['one', 'two']) 

In [ ]:
data2

El apilamiento por “default” filtra los datos faltantes, por lo que la
operación es fácilmente invertible:

In [ ]:
data2.unstack()

In [ ]:
data2.unstack().stack()

In [ ]:
data2.unstack().stack(dropna=False)

Cuando se desapila en un DataFrame, el nivel desapilado se convierte en
el nivel más bajo en el resultado:

In [ ]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                         columns=pd.Index(['left', 'right'], name='side')) 

In [ ]:
df

In [ ]:
df.unstack('state')

Al llamar a `stack`, podemos indicar el nombre del eje a apilar:
 

In [ ]:
df.unstack('state').stack('side')

### Pivotar Formato “Largo” a “Ancho”

 

Una forma común de almacenar múltiples series de tiempo en bases de
datos y CSV es en el formato llamado `long` o `stacked`. Carguemos
algunos datos de ejemplo y realicemos un pa???? pequeña cantidad de manejo
de series de tiempo y limpieza de datos:

In [ ]:
data=pd.read_table("./examples/macrodata.csv", sep='\s+')

In [ ]:
data.head()

In [ ]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                               name='date')

In [ ]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')

In [ ]:
data = data.reindex(columns=columns)

In [ ]:
data.head()

In [ ]:
Out[43  
item   realgdp  infl  unemp
0     2710.349  0.00    5.8
1     2778.801  2.34    5.1
2     2775.488  2.74    5.3
3     2785.204  0.27    5.6
4     2847.699  2.31    5.2


   data.index = periods.to_timestamp('D', 'end')

    ldata = data.stack().reset_index().rename(columns={0: 'value'})
```

 

`PeriodIndex` combina las columnas `year` y `quarter` para crear un tipo
de intervalo de tiempo.

Ahora, `ldata` se ve así:

In [ ]:
data.index = periods.to_timestamp('D', 'end')

In [ ]:
ldata = data.stack().reset_index().rename(columns={0: 'value'})

 

`PeriodIndex` combina las columnas `year` y `quarter` para crear un tipo
de intervalo de tiempo.

Ahora, `ldata` se ve así:

 

 
   

In [ ]:
ldata[:10]

Este es el llamado formato *largo* para múltiples series de tiempo u
otros datos de observación con dos o más claves (aquí, nuestras claves
son `date` y `item`). Cada fila de la tabla representa una sola
observación.

Los datos se almacenan con frecuencia de esta manera en bases de datos
relacionales como MySQL, ya que un esquema fijo —nombres de columna y
tipos de datos— permite que el número de valores distintos en la columna
`item` cambie a medida que se agregan datos a la tabla. En el ejemplo
anterior, `date` y `item` generalmente serían las claves principales —en
el lenguaje de bases de datos relacionales— ofreciendo integridad
relacional y uniones más fáciles. En algunos casos, los datos pueden ser
más difíciles de trabajar en este formato; es posible que prefiera tener
un DataFrame que contenga una columna por cada valor `item` distinto
indexado por marcas de tiempo en la columna `date`. El método pivot de
DataFrame realiza exactamente esta transformación:
 

In [ ]:
pivoted = ldata.pivot('date', 'item', 'value')

In [ ]:
pivoted

In [ ]:
Los primeros dos valores pasados son las columnas que se utilizarán
respectivamente como el índice de fila y columna, luego, finalmente, una
columna de valor opcional para llenar el DataFrame. Supongamos que hay
dos columnas de valor que deseamos remodelar simultáneamente:

Los primeros dos valores pasados son las columnas que se utilizarán
respectivamente como el índice de fila y columna, luego, finalmente, una
columna de valor opcional para llenar el DataFrame. Supongamos que hay
dos columnas de valor que deseamos remodelar simultáneamente:


In [ ]:
ldata['value2'] = np.random.randn(len(ldata))

In [ ]:
ldata[:10]

In [ ]:
Out[52  
                           date     item     value    value2
0 1959-03-31 23:59:59.999999999  realgdp  2710.349 -1.483364
1 1959-03-31 23:59:59.999999999     infl     0.000 -0.024496
2 1959-03-31 23:59:59.999999999    unemp     5.800 -0.377846
3 1959-06-30 23:59:59.999999999  realgdp  2778.801 -0.284949
4 1959-06-30 23:59:59.999999999     infl     2.340  1.384590
5 1959-06-30 23:59:59.999999999    unemp     5.100 -0.392535
6 1959-09-30 23:59:59.999999999  realgdp  2775.488 -2.460320
7 1959-09-30 23:59:59.999999999     infl     2.740  1.363721
8 1959-09-30 23:59:59.999999999    unemp     5.300 -1.221243
9 1959-12-31 23:59:59.999999999  realgdp  2785.204  1.408105
```

 

Al omitir el último argumento, obtenemos un DataFrame con columnas
jerárquicas:


In [ ]:
pivoted = ldata.pivot('date', 'item')  

In [ ]:
pivoted[:5]

In [ ]:
Out[54  
                              value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.024496 -1.483364   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  1.384590 -0.284949   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3  1.363721 -2.460320   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -0.118157  1.408105   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -0.553291 -1.900002   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.377846  
1959-06-30 23:59:59.999999999 -0.392535  
1959-09-30 23:59:59.999999999 -1.221243  
1959-12-31 23:59:59.999999999  1.235613  
1960-03-31 23:59:59.999999999 -1.199842  


    

In [ ]:
pivoted['value'][:5]

In [ ]:
Out[55  
item                           infl   realgdp  unemp
date                                                
1959-03-31 23:59:59.999999999  0.00  2710.349    5.8
1959-06-30 23:59:59.999999999  2.34  2778.801    5.1
1959-09-30 23:59:59.999999999  2.74  2775.488    5.3
1959-12-31 23:59:59.999999999  0.27  2785.204    5.6
1960-03-31 23:59:59.999999999  2.31  2847.699    5.2
```

 

Notemos que `pivot` es equivalente a crear un índice jerárquico usando
`set_index` seguido de una llamada a `unstack`:

 

In [ ]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')

In [ ]:
unstacked[:7]

In [ ]:
Out[57  
                              value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.024496 -1.483364   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  1.384590 -0.284949   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3  1.363721 -2.460320   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -0.118157  1.408105   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -0.553291 -1.900002   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2 -0.587449  0.991692   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6 -1.014435 -1.619160   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.377846  
1959-06-30 23:59:59.999999999 -0.392535  
1959-09-30 23:59:59.999999999 -1.221243  
1959-12-31 23:59:59.999999999  1.235613  
1960-03-31 23:59:59.999999999 -1.199842  
1960-06-30 23:59:59.999999999 -0.070792  
1960-09-30 23:59:59.999999999  1.101305  
```

 

 

 

### Pivotar Formato “Ancho” a “Largo”

 

Una operación inversa para pivotar para DataFrames es `pandas.melt`. En
lugar de transformar una columna en muchas en un nuevo DataFrame, une
varias columnas en una, produciendo un DataFrame que es más largo que la
entrada. Veamos un ejemplo:
 

In [ ]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                         'A': [1, 2, 3],
                         'B': [4, 5, 6],
                         'C': [7, 8, 9]})

  

In [ ]:
df

La columna `'key'` puede ser un indicador de grupo, y las otras columnas
son valores de datos. Al usar `pandas.melt`, debemos indicar qué
columnas —si las hay— son indicadores de grupo. Aquí usamos `'key'` como
el único indicador de grupo:
 

In [ ]:
melted = pd.melt(df, ['key'])

In [ ]:
melted

Usando `pivot`, podemos regresar al formato original:
   

In [ ]:
reshaped = melted.pivot('key', 'variable', 'value')

In [ ]:
reshaped

Dado que el resultado de `pivot` crea un índice de la columna utilizada
como etiquetas de filas, es posible que queramos usar `reset_index` para
mover los datos de regreso a una columna:

In [ ]:
reshaped.reset_index()

También podemos especificar un subconjunto de columnas para usarlas como
columnas de valor:

In [ ]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

`pandas.melt` también se puede usar sin ningún identificador de grupo:

 

In [ ]:
pd.melt(df, value_vars=['A', 'B', 'C'])

In [ ]:
pd.melt(df, value_vars=['key', 'A', 'B'])

 

Ahora que tenemos algunos conceptos básicos de pandas para la
importación, limpieza y reorganización de datos, estamos listos para
pasar a la Agregación de Datos y Operaciones con Grupos.

 

 

 

Bibliografía
------------

 

\[1\] *Python for Data Analysis, Data Wrangling with Pandas, NumPy, and
IPython, Wes McKinney, 2nd. edition, 2018.*

\[2\]
*<a href="https://docs.python.org/3/" class="uri">https://docs.python.org/3/</a>*

\[3\]
*<a href="https://docs.python.org/3/tutorial/index.html" class="uri">https://docs.python.org/3/tutorial/index.html</a>*

\[4\]
*<a href="https://numpy.org/doc/stable/" class="uri">https://numpy.org/doc/stable/</a>*

\[5\]
*<a href="https://pandas.pydata.org/" class="uri">https://pandas.pydata.org/</a>*

 

 

------------------------------------------------------------------------